<a href="https://colab.research.google.com/github/emisoft-designs/SEEFAR-SUSTAIN/blob/academy-data-extraction-v3/SEEFAR_SUSTAIN_(Academy_DB_Extraction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab Notebook – Data Extraction from SEEFAR Academy Database**

In [1]:
!pip install pymysql sqlalchemy gspread gspread-dataframe

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
"""
SUSTAIN MEL Data Extraction & Documentation Script
====================================================

Purpose: Extract, structure, and document Moodle database data for Monitoring,
         Evaluation, and Learning (MEL) indicators for SUSTAIN training programs.

Author: Seefar Academy Data Team
Date: 2026-02-03
Version: 2.0

Database: Seefar Academy Pathways Moodle (pathways_moodle)
Target: MEL reporting for SUSTAIN, Onboarding, and General Preparatory Training

Key Objectives:
1. Map database schema for training delivery, assessments, users, and roles
2. Link Course ID ↔ Course Name for training track filtering
3. Standardize login dates (First Login, Last Login)
4. Extract and relate pre-test/post-test results to users and courses
5. Extract demographics for disaggregation (name, email, phone, gender, country)
6. Differentiate user types (Participants, Candidates, Employers, Visa Facilitators)
7. Support MEL indicator calculations (completion rates, knowledge increase, etc.)
8. Enable automated reporting and dashboarding
"""

# ============================================================================
# SECTION 1: SETUP & CONFIGURATION
# ============================================================================

# Install required packages
# !pip install pymysql sqlalchemy gspread gspread-dataframe pandas numpy
import os
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, text
from typing import Dict, List, Tuple, Any, Optional
import warnings
from datetime import datetime
import json
import time

warnings.filterwarnings('ignore')

# Database credentials configuration
class DatabaseConfig:
    """Database connection configuration"""
    DB_USER = "dbuserread"
    # DB_PASSWORD should be set via environment variable or Google Colab userdata
    DB_HOST = "ls-17122ea7e3a528fd292a260b6217b006cb7a0f38.cyhh372xsm2h.ap-southeast-1.rds.amazonaws.com"
    DB_PORT = 3306
    DB_NAME = "pathways_moodle"

    @classmethod
    def get_connection_url(cls, password: str) -> str:
        """Generate database connection URL"""
        return f"mysql+pymysql://{cls.DB_USER}:{password}@{cls.DB_HOST}:{cls.DB_PORT}/{cls.DB_NAME}"


# ============================================================================
# SECTION 2: DATABASE UTILITY FUNCTIONS
# ============================================================================

def create_db_engine(password: str):
    """
    Create SQLAlchemy engine for database connection.

    Args:
        password: Database password

    Returns:
        SQLAlchemy engine object
    """
    connection_url = DatabaseConfig.get_connection_url(password)
    engine = create_engine(connection_url, pool_pre_ping=True)
    print("✓ Database engine created successfully")
    return engine


def fetch_data(engine, query: str, params: Optional[Dict] = None) -> pd.DataFrame:
    """
    Execute SQL query and return results as DataFrame.

    Args:
        engine: SQLAlchemy engine
        query: SQL query string
        params: Optional query parameters

    Returns:
        pandas DataFrame with query results
    """
    try:
        with engine.connect() as conn:
            if params:
                df = pd.read_sql(text(query), conn, params=params)
            else:
                df = pd.read_sql(query, conn)
        print(f"✓ Fetched {df.shape[0]} rows × {df.shape[1]} columns")
        return df
    except Exception as e:
        print(f"✗ Error fetching data: {e}")
        return pd.DataFrame()


def get_table_info(engine, table_name: str) -> pd.DataFrame:
    """
    Get column information for a specific table.

    Args:
        engine: SQLAlchemy engine
        table_name: Name of the table

    Returns:
        DataFrame with column information
    """
    query = f"""
    SELECT
        COLUMN_NAME as column_name,
        DATA_TYPE as data_type,
        IS_NULLABLE as is_nullable,
        COLUMN_KEY as column_key,
        COLUMN_COMMENT as column_comment
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_SCHEMA = '{DatabaseConfig.DB_NAME}'
    AND TABLE_NAME = '{table_name}'
    ORDER BY ORDINAL_POSITION
    """
    return fetch_data(engine, query)


# ============================================================================
# SECTION 3: CORE DATA EXTRACTION FUNCTIONS
# ============================================================================

class MELDataExtractor:
    """Main class for extracting MEL-relevant data from Moodle database"""

    def __init__(self, engine):
        self.engine = engine
        self.extracted_data = {}

    def extract_users(self) -> pd.DataFrame:
        """
        Extract user data with demographic information.

        Returns:
            DataFrame with user information including demographics
        """
        print("\n" + "="*80)
        print("EXTRACTING USER DATA")
        print("="*80)

        query = """
        SELECT
            u.id as user_id,
            u.username,
            u.firstname,
            u.lastname,
            u.email,
            u.phone1 as phone,
            u.country,
            u.city,
            u.institution,
            u.department,
            u.timecreated as user_created_timestamp,
            u.timemodified as user_modified_timestamp,
            u.firstaccess as first_login_timestamp,
            u.lastaccess as last_login_timestamp,
            u.lastlogin as last_login_timestamp_alt,
            u.currentlogin as current_login_timestamp,
            u.confirmed,
            u.suspended,
            u.deleted,
            FROM_UNIXTIME(u.timecreated) as user_created_date,
            FROM_UNIXTIME(u.firstaccess) as first_login_date,
            FROM_UNIXTIME(u.lastaccess) as last_login_date
        FROM mdl_user u
        WHERE u.deleted = 0
        AND u.id > 2  -- Exclude guest and admin system accounts
        ORDER BY u.id
        """

        users_df = fetch_data(self.engine, query)

        # Add custom fields (demographics)
        demographics_query = """
        SELECT
            uid.userid as user_id,
            uif.shortname as field_name,
            uid.data as field_value
        FROM mdl_user_info_data uid
        JOIN mdl_user_info_field uif ON uid.fieldid = uif.id
        WHERE uif.shortname IN ('gender', 'age', 'agegroup', 'nationality', 'ethnicity')
        """

        demographics_df = fetch_data(self.engine, demographics_query)

        # Pivot demographics data
        if not demographics_df.empty:
            demographics_pivot = demographics_df.pivot(
                index='user_id',
                columns='field_name',
                values='field_value'
            ).reset_index()

            users_df = users_df.merge(demographics_pivot, on='user_id', how='left')

        print(f"✓ Extracted {len(users_df)} users with demographics")
        self.extracted_data['users'] = users_df
        return users_df

    def extract_courses(self) -> pd.DataFrame:
        """
        Extract course information with categorization.

        Returns:
            DataFrame with course details
        """
        print("\n" + "="*80)
        print("EXTRACTING COURSE DATA")
        print("="*80)

        query = """
        SELECT
            c.id as course_id,
            c.category as course_category_id,
            c.fullname as course_name,
            c.shortname as course_shortname,
            c.summary as course_description,
            c.startdate as course_start_timestamp,
            c.enddate as course_end_timestamp,
            FROM_UNIXTIME(c.startdate) as course_start_date,
            FROM_UNIXTIME(c.enddate) as course_end_date,
            c.visible as is_visible,
            c.timecreated as course_created_timestamp,
            FROM_UNIXTIME(c.timecreated) as course_created_date,
            cc.name as category_name,
            cc.path as category_path
        FROM mdl_course c
        LEFT JOIN mdl_course_categories cc ON c.category = cc.id
        WHERE c.id > 1  -- Exclude site home course
        ORDER BY c.id
        """

        courses_df = fetch_data(self.engine, query)

        # Categorize courses based on name patterns
        def categorize_course(row):
            """Categorize course into training tracks"""
            course_name = str(row['course_name']).lower()
            course_short = str(row['course_shortname']).lower()

            if 'sustain' in course_name or 'sustain' in course_short:
                return 'SUSTAIN Training'
            elif 'onboard' in course_name or 'onboard' in course_short:
                return 'Onboarding Training'
            elif 'prep' in course_name or 'career' in course_name or 'stem' in course_name:
                return 'General Preparatory Training'
            elif 'employer' in course_name:
                return 'Employer Training'
            elif 'visa' in course_name or 'facilitator' in course_name:
                return 'Visa Facilitator Training'
            else:
                return 'Other'

        courses_df['training_track'] = courses_df.apply(categorize_course, axis=1)

        print(f"✓ Extracted {len(courses_df)} courses")
        print("\nCourse distribution by training track:")
        print(courses_df['training_track'].value_counts())

        self.extracted_data['courses'] = courses_df
        return courses_df

    def extract_user_roles(self) -> pd.DataFrame:
        """
        Extract user role assignments to differentiate user types.

        Returns:
            DataFrame with user roles
        """
        print("\n" + "="*80)
        print("EXTRACTING USER ROLES")
        print("="*80)

        query = """
        SELECT
            ra.id as role_assignment_id,
            ra.userid as user_id,
            ra.roleid as role_id,
            r.shortname as role_shortname,
            r.name as role_name,
            ra.contextid,
            c.contextlevel,
            c.instanceid,
            FROM_UNIXTIME(ra.timemodified) as role_assigned_date
        FROM mdl_role_assignments ra
        JOIN mdl_role r ON ra.roleid = r.id
        JOIN mdl_context c ON ra.contextid = c.id
        ORDER BY ra.userid, ra.roleid
        """

        roles_df = fetch_data(self.engine, query)

        # Categorize user types based on roles
        def categorize_user_type(role_shortname):
            """Map role to user type category"""
            role_lower = str(role_shortname).lower()

            if 'student' in role_lower:
                return 'Participant/Candidate'
            elif 'teacher' in role_lower or 'editingteacher' in role_lower:
                return 'Facilitator/Instructor'
            elif 'employer' in role_lower:
                return 'Employer'
            elif 'manager' in role_lower:
                return 'Manager'
            else:
                return 'Other'

        roles_df['user_type_category'] = roles_df['role_shortname'].apply(categorize_user_type)

        print(f"✓ Extracted {len(roles_df)} role assignments")
        print("\nUser type distribution:")
        print(roles_df['user_type_category'].value_counts())

        self.extracted_data['user_roles'] = roles_df
        return roles_df

    def extract_enrollments(self) -> pd.DataFrame:
        """
        Extract user course enrollments.

        Returns:
            DataFrame with enrollment data
        """
        print("\n" + "="*80)
        print("EXTRACTING ENROLLMENT DATA")
        print("="*80)

        query = """
        SELECT
            ue.id as enrollment_id,
            ue.userid as user_id,
            ue.enrolid,
            e.courseid as course_id,
            e.enrol as enrollment_method,
            ue.status as enrollment_status,
            ue.timestart as enrollment_start_timestamp,
            ue.timeend as enrollment_end_timestamp,
            ue.timecreated as enrollment_created_timestamp,
            ue.timemodified as enrollment_modified_timestamp,
            FROM_UNIXTIME(ue.timecreated) as enrollment_date,
            FROM_UNIXTIME(ue.timestart) as enrollment_start_date,
            FROM_UNIXTIME(ue.timeend) as enrollment_end_date
        FROM mdl_user_enrolments ue
        JOIN mdl_enrol e ON ue.enrolid = e.id
        ORDER BY ue.userid, e.courseid
        """

        enrollments_df = fetch_data(self.engine, query)

        # Add enrollment status label
        enrollments_df['enrollment_status_label'] = enrollments_df['enrollment_status'].map({
            0: 'Active',
            1: 'Suspended'
        })

        print(f"✓ Extracted {len(enrollments_df)} enrollments")

        self.extracted_data['enrollments'] = enrollments_df
        return enrollments_df

    def extract_quiz_data(self) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
        """
        Extract quiz, quiz attempts, and quiz grades for pre/post-test analysis.

        Returns:
            Tuple of (quizzes, attempts, grades) DataFrames
        """
        print("\n" + "="*80)
        print("EXTRACTING QUIZ/ASSESSMENT DATA")
        print("="*80)

        # Quizzes
        quiz_query = """
        SELECT
            q.id as quiz_id,
            q.course as course_id,
            q.name as quiz_name,
            q.intro as quiz_description,
            q.timeopen as quiz_open_timestamp,
            q.timeclose as quiz_close_timestamp,
            q.timelimit as time_limit_seconds,
            q.grade as max_grade,
            q.sumgrades as sum_grades,
            FROM_UNIXTIME(q.timeopen) as quiz_open_date,
            FROM_UNIXTIME(q.timeclose) as quiz_close_date
        FROM mdl_quiz q
        ORDER BY q.course, q.id
        """
        quizzes_df = fetch_data(self.engine, quiz_query)

        # Categorize quizzes as pre-test or post-test
        def categorize_quiz(quiz_name):
            """Identify pre-test vs post-test"""
            name_lower = str(quiz_name).lower()
            if 'pre' in name_lower or 'pre-test' in name_lower or 'pretest' in name_lower:
                return 'Pre-Test'
            elif 'post' in name_lower or 'post-test' in name_lower or 'posttest' in name_lower:
                return 'Post-Test'
            else:
                return 'Assessment'

        quizzes_df['assessment_type'] = quizzes_df['quiz_name'].apply(categorize_quiz)

        # Quiz attempts
        attempts_query = """
        SELECT
            qa.id as attempt_id,
            qa.quiz as quiz_id,
            qa.userid as user_id,
            qa.attempt as attempt_number,
            qa.state as attempt_state,
            qa.timestart as attempt_start_timestamp,
            qa.timefinish as attempt_finish_timestamp,
            qa.timemodified as attempt_modified_timestamp,
            qa.sumgrades as attempt_score,
            FROM_UNIXTIME(qa.timestart) as attempt_start_date,
            FROM_UNIXTIME(qa.timefinish) as attempt_finish_date,
            TIMESTAMPDIFF(SECOND, FROM_UNIXTIME(qa.timestart), FROM_UNIXTIME(qa.timefinish)) as duration_seconds
        FROM mdl_quiz_attempts qa
        WHERE qa.state = 'finished'
        ORDER BY qa.userid, qa.quiz, qa.attempt
        """
        attempts_df = fetch_data(self.engine, attempts_query)

        # Quiz grades
        grades_query = """
        SELECT
            qg.id as grade_id,
            qg.quiz as quiz_id,
            qg.userid as user_id,
            qg.grade as final_grade,
            qg.timemodified as grade_timestamp,
            FROM_UNIXTIME(qg.timemodified) as grade_date
        FROM mdl_quiz_grades qg
        ORDER BY qg.userid, qg.quiz
        """
        grades_df = fetch_data(self.engine, grades_query)

        print(f"✓ Extracted {len(quizzes_df)} quizzes")
        print(f"✓ Extracted {len(attempts_df)} quiz attempts")
        print(f"✓ Extracted {len(grades_df)} quiz grades")
        print("\nQuiz type distribution:")
        print(quizzes_df['assessment_type'].value_counts())

        self.extracted_data['quizzes'] = quizzes_df
        self.extracted_data['quiz_attempts'] = attempts_df
        self.extracted_data['quiz_grades'] = grades_df

        return quizzes_df, attempts_df, grades_df

    def extract_assignment_data(self) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
        """
        Extract assignment, submission, and grading data.

        Returns:
            Tuple of (assignments, submissions, grades) DataFrames
        """
        print("\n" + "="*80)
        print("EXTRACTING ASSIGNMENT DATA")
        print("="*80)

        # Assignments
        assign_query = """
        SELECT
            a.id as assignment_id,
            a.course as course_id,
            a.name as assignment_name,
            a.intro as assignment_description,
            a.duedate as due_timestamp,
            a.allowsubmissionsfromdate as submissions_from_timestamp,
            a.grade as max_grade,
            FROM_UNIXTIME(a.duedate) as due_date,
            FROM_UNIXTIME(a.allowsubmissionsfromdate) as submissions_from_date
        FROM mdl_assign a
        ORDER BY a.course, a.id
        """
        assignments_df = fetch_data(self.engine, assign_query)

        # Submissions
        submissions_query = """
        SELECT
            asub.id as submission_id,
            asub.assignment as assignment_id,
            asub.userid as user_id,
            asub.status as submission_status,
            asub.timecreated as submission_created_timestamp,
            asub.timemodified as submission_modified_timestamp,
            asub.attemptnumber as attempt_number,
            asub.latest as is_latest,
            FROM_UNIXTIME(asub.timecreated) as submission_created_date,
            FROM_UNIXTIME(asub.timemodified) as submission_modified_date
        FROM mdl_assign_submission asub
        ORDER BY asub.userid, asub.assignment
        """
        submissions_df = fetch_data(self.engine, submissions_query)

        # Grades
        assign_grades_query = """
        SELECT
            ag.id as grade_id,
            ag.assignment as assignment_id,
            ag.userid as user_id,
            ag.grade as grade,
            ag.grader as grader_id,
            ag.attemptnumber as attempt_number,
            ag.timecreated as grade_created_timestamp,
            ag.timemodified as grade_modified_timestamp,
            FROM_UNIXTIME(ag.timecreated) as grade_created_date,
            FROM_UNIXTIME(ag.timemodified) as grade_modified_date
        FROM mdl_assign_grades ag
        WHERE ag.grade >= 0  -- Exclude ungraded submissions
        ORDER BY ag.userid, ag.assignment
        """
        assign_grades_df = fetch_data(self.engine, assign_grades_query)

        print(f"✓ Extracted {len(assignments_df)} assignments")
        print(f"✓ Extracted {len(submissions_df)} submissions")
        print(f"✓ Extracted {len(assign_grades_df)} assignment grades")

        self.extracted_data['assignments'] = assignments_df
        self.extracted_data['assignment_submissions'] = submissions_df
        self.extracted_data['assignment_grades'] = assign_grades_df

        return assignments_df, submissions_df, assign_grades_df

    def extract_feedback_data(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Extract feedback/survey responses for training usefulness assessment.

        Returns:
            Tuple of (feedback_activities, responses) DataFrames
        """
        print("\n" + "="*80)
        print("EXTRACTING FEEDBACK/SURVEY DATA")
        print("="*80)

        # Feedback activities
        feedback_query = """
        SELECT
            f.id as feedback_id,
            f.course as course_id,
            f.name as feedback_name,
            f.intro as feedback_description,
            f.timeopen as feedback_open_timestamp,
            f.timeclose as feedback_close_timestamp,
            FROM_UNIXTIME(f.timeopen) as feedback_open_date,
            FROM_UNIXTIME(f.timeclose) as feedback_close_date
        FROM mdl_feedback f
        ORDER BY f.course, f.id
        """
        feedback_df = fetch_data(self.engine, feedback_query)

        # Completed responses
        responses_query = """
        SELECT
            fc.id as response_id,
            fc.feedback as feedback_id,
            fc.userid as user_id,
            fc.timemodified as response_timestamp,
            FROM_UNIXTIME(fc.timemodified) as response_date
        FROM mdl_feedback_completed fc
        ORDER BY fc.userid, fc.feedback
        """
        responses_df = fetch_data(self.engine, responses_query)

        # Questionnaire data
        questionnaire_query = """
        SELECT
            qr.id as questionnaire_response_id,
            qr.questionnaireid as questionnaire_id,
            qr.userid as user_id,
            qr.submitted as submission_timestamp,
            qr.complete as is_complete,
            FROM_UNIXTIME(qr.submitted) as submission_date
        FROM mdl_questionnaire_response qr
        ORDER BY qr.userid, qr.questionnaireid
        """
        questionnaire_df = fetch_data(self.engine, questionnaire_query)

        print(f"✓ Extracted {len(feedback_df)} feedback activities")
        print(f"✓ Extracted {len(responses_df)} feedback responses")
        print(f"✓ Extracted {len(questionnaire_df)} questionnaire responses")

        self.extracted_data['feedback'] = feedback_df
        self.extracted_data['feedback_responses'] = responses_df
        self.extracted_data['questionnaire_responses'] = questionnaire_df

        return feedback_df, responses_df

    def extract_completion_data(self) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
        """
        Extract course and module completion data.

        Returns:
            Tuple of (course_completions, module_completions, certificates) DataFrames
        """
        print("\n" + "="*80)
        print("EXTRACTING COMPLETION DATA")
        print("="*80)

        # Course completions
        course_completion_query = """
        SELECT
            cc.id as completion_id,
            cc.userid as user_id,
            cc.course as course_id,
            cc.timeenrolled as enrollment_timestamp,
            cc.timestarted as started_timestamp,
            cc.timecompleted as completed_timestamp,
            cc.reaggregate,
            FROM_UNIXTIME(cc.timeenrolled) as enrollment_date,
            FROM_UNIXTIME(cc.timestarted) as started_date,
            FROM_UNIXTIME(cc.timecompleted) as completed_date
        FROM mdl_course_completions cc
        WHERE cc.timecompleted IS NOT NULL
        ORDER BY cc.userid, cc.course
        """
        course_completions_df = fetch_data(self.engine, course_completion_query)

        # Module completions
        module_completion_query = """
        SELECT
            cmc.id as module_completion_id,
            cmc.coursemoduleid as course_module_id,
            cmc.userid as user_id,
            cmc.completionstate as completion_state,
            cmc.timemodified as completion_timestamp,
            FROM_UNIXTIME(cmc.timemodified) as completion_date
        FROM mdl_course_modules_completion cmc
        WHERE cmc.completionstate > 0
        ORDER BY cmc.userid, cmc.coursemoduleid
        """
        module_completions_df = fetch_data(self.engine, module_completion_query)

        # Map completion states
        module_completions_df['completion_state_label'] = module_completions_df['completion_state'].map({
            1: 'Complete',
            2: 'Complete (Pass)',
            3: 'Complete (Fail)'
        })

        # Certificates issued
        certificates_query = """
        SELECT
            ci.id as certificate_issue_id,
            ci.userid as user_id,
            ci.customcertid as certificate_id,
            ci.code as certificate_code,
            ci.timecreated as issued_timestamp,
            FROM_UNIXTIME(ci.timecreated) as issued_date
        FROM mdl_customcert_issues ci
        ORDER BY ci.userid, ci.customcertid
        """
        certificates_df = fetch_data(self.engine, certificates_query)

        print(f"✓ Extracted {len(course_completions_df)} course completions")
        print(f"✓ Extracted {len(module_completions_df)} module completions")
        print(f"✓ Extracted {len(certificates_df)} certificates issued")

        self.extracted_data['course_completions'] = course_completions_df
        self.extracted_data['module_completions'] = module_completions_df
        self.extracted_data['certificates'] = certificates_df

        return course_completions_df, module_completions_df, certificates_df

    def extract_activity_logs(self, limit: int = 100000) -> pd.DataFrame:
        """
        Extract user activity logs for engagement analysis.

        Args:
            limit: Maximum number of log entries to retrieve

        Returns:
            DataFrame with activity log data
        """
        print("\n" + "="*80)
        print("EXTRACTING ACTIVITY LOGS")
        print("="*80)

        query = f"""
        SELECT
            l.id as log_id,
            l.userid as user_id,
            l.courseid as course_id,
            l.eventname,
            l.component,
            l.action,
            l.target,
            l.timecreated as event_timestamp,
            FROM_UNIXTIME(l.timecreated) as event_date,
            l.origin,
            l.ip
        FROM mdl_logstore_standard_log l
        WHERE l.userid > 2  -- Exclude system users
        ORDER BY l.timecreated DESC
        LIMIT {limit}
        """

        logs_df = fetch_data(self.engine, query)

        print(f"✓ Extracted {len(logs_df)} activity log entries")

        self.extracted_data['activity_logs'] = logs_df
        return logs_df



    @staticmethod
    def extract_first_span_content(html_string):
        """
        Extract meaningful visible text from HTML string.
        - If it starts with a <span>, returns the first span’s text
        - Otherwise returns all visible text joined cleanly
        """
        if pd.isna(html_string):
            return None
        if not isinstance(html_string, str):
            return html_string

        soup = BeautifulSoup(html_string, "html.parser")

        # If HTML has one or more <span>, return first span text
        first_span = soup.find("span")
        if first_span and first_span.text.strip():
            return first_span.text.strip()

        # Otherwise collect all visible text
        texts = [t.strip() for t in soup.stripped_strings if t.strip()]
        return " ".join(texts) if texts else ""

    def extract_all_data(self):
        """Execute all extraction functions"""
        print("\n" + "="*80)
        print("STARTING COMPREHENSIVE DATA EXTRACTION")
        print("="*80)

        self.extract_users()
        self.extract_courses()
        self.extract_user_roles()
        self.extract_enrollments()
        self.extract_quiz_data()
        self.extract_assignment_data()
        self.extract_feedback_data()
        self.extract_completion_data()
        self.extract_activity_logs()

        # Clean the columns
        for name, df in self.extracted_data.items():
            self.extracted_data[name] = df.applymap(self.extract_first_span_content)

        print("\n" + "="*80)
        print("DATA EXTRACTION COMPLETE")
        print("="*80)
        print(f"\nTotal datasets extracted: {len(self.extracted_data)}")
        print("\nDataset summary:")
        for name, df in self.extracted_data.items():
            print(f"  - {name}: {len(df)} records")

# ============================================================================
# SECTION 5: GOOGLE SHEETS EXPORT
# ============================================================================

def export_to_google_sheets(
    extracted_data: Dict[str, pd.DataFrame],
    spreadsheet_name: str = "SUSTAIN MEL Data Export (RAW)",
    folder_name: str = "SUSTAIN_ACADEMY",
):
    import time
    from google.colab import auth
    import gspread
    from gspread_dataframe import set_with_dataframe
    from google.auth import default
    from googleapiclient.discovery import build

    auth.authenticate_user()
    creds, _ = default(scopes=[
        "https://www.googleapis.com/auth/drive",
        "https://www.googleapis.com/auth/spreadsheets"
    ])
    gc = gspread.authorize(creds)
    drive = build("drive", "v3", credentials=creds)

    # find or create folder
    q = (
        "mimeType='application/vnd.google-apps.folder' "
        f"and name='{folder_name}' and trashed=false"
    )
    res = drive.files().list(q=q, fields="files(id,name)").execute()
    folder = res.get("files", [])
    if folder:
        folder_id = folder[0]["id"]
    else:
        fld = {"name": folder_name, "mimeType": "application/vnd.google-apps.folder"}
        folder_id = drive.files().create(body=fld, fields="id").execute()["id"]

    # find existing spreadsheet
    q = (
        "mimeType='application/vnd.google-apps.spreadsheet' "
        f"and name='{spreadsheet_name}' and '{folder_id}' in parents and trashed=false"
    )
    res = drive.files().list(q=q, fields="files(id,name)").execute()
    files = res.get("files", [])

    if files:
        ss_id = files[0]["id"]
        spreadsheet = gc.open_by_key(ss_id)
        print(f"Opening existing: {spreadsheet.url}")
    else:
        spreadsheet = gc.create(spreadsheet_name)
        file_id = spreadsheet.id
        drive.files().update(
            fileId=file_id,
            addParents=folder_id,
            removeParents="root",
            fields="id, parents"
        ).execute()
        print(f"Created new: {spreadsheet.url}")

    # write or overwrite sheets
    for sheet_name, df in extracted_data.items():
        if df.empty:
            continue

        title = sheet_name[:100]
        try:
            try:
                ws = spreadsheet.worksheet(title)
                spreadsheet.del_worksheet(ws)
            except:
                pass

            ws = spreadsheet.add_worksheet(
                title=title,
                rows=max(len(df) + 1, 2),
                cols=max(len(df.columns), 1),
            )
            set_with_dataframe(ws, df)
            time.sleep(1)
            print(f"Exported {title}")
        except Exception as e:
            print(f"Error {title}: {e}")

    return spreadsheet.url


# ============================================================================
# SECTION 7: MAIN EXECUTION
# ============================================================================

def main():
    """Main execution function"""

    # Step 1: Get database password
    try:
        from google.colab import userdata
        db_password = userdata.get('db_password')
        print("✓ Retrieved database password from Colab secrets")
    except:
        # For local testing, use environment variable
        import os
        db_password = os.getenv('DB_PASSWORD')
        if not db_password:
            print("✗ Database password not found!")
            print("  Set 'db_password' in Colab secrets or DB_PASSWORD environment variable")
            return

    # Step 2: Create database engine
    engine = create_db_engine(db_password)

    # Step 3: Extract all data
    extractor = MELDataExtractor(engine)
    extractor.extract_all_data()

    # Step 4: Calculate MEL indicators
    calculator = MELIndicatorCalculator(extractor.extracted_data)

    # Step 6: Export to Google Sheets
    all_data = {**extractor.extracted_data}
    sheets_url = export_to_google_sheets(all_data)

    # Step 7: Print summary
    print("\n" + "="*80)
    print("EXECUTION COMPLETE")
    print("="*80)
    print(f"""
Summary:
  - Total datasets extracted: {len(extractor.extracted_data)}
  - Documentation generated: MEL_Data_Documentation.md
  - Google Sheets URL: {sheets_url if sheets_url else 'Export failed'}

Next Steps:
  1. Review the Google Sheets export
  2. Read the documentation file
  3. Set up automated dashboards
  4. Configure scheduled reporting

For support, refer to the documentation or contact the data team.
""")

    return {
        'extracted_data': extractor.extracted_data,
        'indicators': calculator.indicators,
        'sheets_url': sheets_url
    }


# ============================================================================
# EXECUTE
# ============================================================================


if __name__ == "__main__":
    results = main()


✓ Retrieved database password from Colab secrets
✓ Database engine created successfully

STARTING COMPREHENSIVE DATA EXTRACTION

EXTRACTING USER DATA
✓ Fetched 2014 rows × 22 columns
✓ Fetched 5019 rows × 3 columns
✓ Extracted 2014 users with demographics

EXTRACTING COURSE DATA
✓ Fetched 30 rows × 14 columns
✓ Extracted 30 courses

Course distribution by training track:
training_track
Other                           29
General Preparatory Training     1
Name: count, dtype: int64

EXTRACTING USER ROLES
✓ Fetched 1506 rows × 9 columns
✓ Extracted 1506 role assignments

User type distribution:
user_type_category
Participant/Candidate     1471
Other                       16
Manager                     12
Facilitator/Instructor       7
Name: count, dtype: int64

EXTRACTING ENROLLMENT DATA
✓ Fetched 1483 rows × 13 columns
✓ Extracted 1483 enrollments

EXTRACTING QUIZ/ASSESSMENT DATA
✓ Fetched 301 rows × 11 columns
✓ Fetched 6960 rows × 12 columns
✓ Fetched 5354 rows × 6 columns
✓ Extracted 


DATA EXTRACTION COMPLETE

Total datasets extracted: 17

Dataset summary:
  - users: 2014 records
  - courses: 30 records
  - user_roles: 1506 records
  - enrollments: 1483 records
  - quizzes: 301 records
  - quiz_attempts: 6960 records
  - quiz_grades: 5354 records
  - assignments: 4 records
  - assignment_submissions: 55 records
  - assignment_grades: 0 records
  - feedback: 64 records
  - feedback_responses: 1378 records
  - questionnaire_responses: 3064 records
  - course_completions: 372 records
  - module_completions: 20102 records
  - certificates: 564 records
  - activity_logs: 100000 records
Opening existing: https://docs.google.com/spreadsheets/d/1rF_HoRlOUPjBEnmyIo7wy-MGgQ0CB0Pbli3j9IU2PaQ
Exported users
Exported courses
Exported user_roles
Exported enrollments
Exported quizzes
Exported quiz_attempts
Exported quiz_grades
Exported assignments
Exported assignment_submissions
Exported feedback
Exported feedback_responses
Exported questionnaire_responses
Exported course_complet